In [1]:
from pylammpsmpi.lammps import LammpsLibrary

In [2]:
lmp = LammpsLibrary(cores=2)

Set up system

In [3]:
lmp.file("in.simple")

In [4]:
lmp.extract_setting("units")

-1

In [5]:
lmp.extract_atom("f")

[[-0.255291613596607, 4.609419196795491, 11.488556813439391],
 [16.37266096862701, -7.1677713665076634, -16.490738409506925],
 [4.159322561393292, 3.049756344619408, -1.672576432988547],
 [-10.358734928539349, -9.449034361720962, 19.63506702653769],
 [-0.9411205246407262, 1.2692762413238663, 0.8581300952563555],
 [2.938717474528063, 1.8778322363819204, 4.395434974329458],
 [-4.466552742048257, 14.313284762555803, 9.992931965435597],
 [3.300931493929359, -9.62076771682111, -10.179346921225509],
 [7.504103848449664, -3.438408604497025, 2.037951963031357],
 [0.32879193591527645, -14.938783633984462, -5.241613989842492],
 [-0.9406476961963371, 8.727601250384891, -5.074895285279922],
 [-4.245722284654334, 0.5935184460072933, -2.4007094062329983],
 [6.018469438850848, 3.805620128400143, 4.4155719577393775],
 [6.224548675112736, -6.306987567024483, 15.75014042448029],
 [-1.3057190929343863, 0.5953631162161415, 1.3737007156747547],
 [-4.54408845092879, -9.172737139679425, -12.979185544829804],

In [ ]:
lmp.gather_atoms("f")

In [ ]:
# extract force on single atom two different ways

f = lmp.extract_atom("f",3)
print("Force on 1 atom via extract_atom: ",f[0][0])

fx = lmp.extract_variable("fx","all",1)
print("Force on 1 atom via extract_variable:",fx[0])

In [ ]:
# use commands_string() and commands_list() to invoke more commands

strtwo = "run 10\nrun 20"
lmp.commands_string(strtwo)

cmds = ["run 10","run 20"]
lmp.commands_list(cmds)

In [ ]:
# delete all atoms
# create_atoms() to create new ones with old coords, vels
# initial thermo should be same as step 20

natoms = lmp.get_natoms()
atype = natoms*[1]

lmp.command("delete_atoms group all");
lmp.create_atoms(natoms,None,atype,x,v);
lmp.command("run 10");

In [ ]:
############
# test of new gather/scatter and box extract/reset methods
# can try this in parallel and with/without atom_modify sort enabled

lmp.command("write_dump all custom tmp.simple id type x y z fx fy fz");

x = lmp.gather_atoms("x",1,3)
f = lmp.gather_atoms("f",1,3)

if me == 0: print("Gather XF:",x[3],x[9],f[3],f[9])

ids = lmp.gather_atoms_concat("id",0,1)
x = lmp.gather_atoms_concat("x",1,3)
f = lmp.gather_atoms_concat("f",1,3)

if me == 0: print("Gather concat XF:",ids[0],ids[1],x[0],x[3],f[0],f[3])

ids = (2*ctypes.c_int)()
ids[0] = 2
ids[1] = 4
x = lmp.gather_atoms_subset("x",1,3,2,ids)
f = lmp.gather_atoms_subset("f",1,3,2,ids)

if me == 0: print("Gather subset XF:",x[0],x[3],f[0],f[3])

x[0] = -1.0
x[1] = 0.0
x[2] = 0.0
x[3] = -2.0
x[4] = 0.0
x[5] = 0.0
ids[0] = 100
ids[1] = 200
lmp.scatter_atoms_subset("x",1,3,2,ids,x)

x = lmp.gather_atoms("x",1,3)
if me == 0: print("Gather post scatter subset:",
                  x[3],x[9],x[297],x[298],x[299],x[597],x[598],x[599])

boxlo,boxhi,xy,yz,xz,periodicity,box_change = lmp.extract_box()
if me == 0: print("Box info",boxlo,boxhi,xy,yz,xz,periodicity,box_change)

lmp.reset_box([0,0,0],[10,10,8],0,0,0)

boxlo,boxhi,xy,yz,xz,periodicity,box_change = lmp.extract_box()
if me == 0: print("Box info",boxlo,boxhi,xy,yz,xz,periodicity,box_change)